# Associate Ca2+ signal with position on cheeseboard for each session using crossregistration

Define Experiment type

In [ ]:
all_expe_types =['Cheeseboard']

dir = "//10.69.168.1/crnldata/forgetting/Aurelie/MiniscopeOE_data/L2_3_mice/"

In [ ]:
cd "C:/Users/Manip2/SCRIPTS/minian/"

Load packages

In [ ]:
import os
import numpy as np
from scipy import signal
import quantities as pq
import math 
import neo
import json
from pathlib import Path
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import pickle
import sys 
from datetime import datetime
import shutil
from ast import literal_eval
from scipy.signal import find_peaks
from scipy.stats import zscore
from scipy import stats
from itertools import groupby
from IPython.display import display
from scipy.interpolate import griddata
import numpy as np
from scipy.signal import butter, filtfilt, hilbert
import matplotlib.pyplot as plt
from scipy import interpolate
import numpy.matlib
from sklearn.decomposition import PCA
from scipy import stats
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import resample
from scipy.signal import resample_poly
from math import gcd
import warnings
warnings.filterwarnings("ignore")
import sys
from scipy.interpolate import interp1d
from collections import defaultdict
import bisect
from scipy.ndimage import gaussian_filter
import random
from scipy.ndimage import gaussian_filter1d
from matplotlib.collections import LineCollection

class Tee:
    def __init__(self, *files):
        self.files = files
    def write(self, obj):
        for f in self.files:
            f.write(obj)
            f.flush()
    def flush(self):
        for f in self.files:
            f.flush()


minian_path = os.path.join(os.path.abspath('.'),'minian')
print("The folder used for minian procedures is : {}".format(minian_path))
sys.path.append(minian_path)

from minian.utilities import (
    TaskAnnotation,
    get_optimal_chk,
    load_videos,
    open_minian,
    save_minian,
)

Define parameters

In [ ]:
pixel_to_cm = 2.25  
table_center_x, table_center_y = 313, 283  # Center of the cheeseboard table on the video
table_center_x, table_center_y = 300, 270  # Center of the cheeseboard table on the video
table_radius = 290 / 2
square_size = pixel_to_cm * 6

bin_size = 10
bin_edges = np.arange(0, 361, bin_size)
bin_centers = np.radians(bin_edges[:-1] + bin_size/2)

Define functions

In [ ]:
def remove_outliers_avg_filter(data):
    data = np.array(data, dtype=float)  # Ensure NumPy array with float type
    filtered_data = np.copy(data)  # Copy to avoid modifying original data
    for i in range(len(data)):
        if not np.isnan(data[i]):  # Skip valid values
            continue
        # Find the closest previous non-NaN value
        prev_idx = i - 1
        while prev_idx >= 0 and np.isnan(data[prev_idx]):
            prev_idx -= 1        
        # Find the closest next non-NaN value
        next_idx = i + 1
        while next_idx < len(data) and np.isnan(data[next_idx]):
            next_idx += 1
        # Compute average if both values exist
        if prev_idx >= 0 and next_idx < len(data):
            filtered_data[i] = (data[prev_idx] + data[next_idx]) / 2
        # If neither exists, NaN remains
    return filtered_data

def find_closest_index_sorted(arr, target):
    idx = bisect.bisect_left(arr, target)  # Find the insertion point
    if idx == 0:
        return 0
    if idx == len(arr):
        return len(arr) - 1
    before = idx - 1
    after = idx
    return before if abs(arr[before] - target) <= abs(arr[after] - target) else after

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

def detect_longest_lowest_sequence(arr, margin=0):
    min_val = np.nanmin(arr)  # Find minimum value
    threshold = min_val + margin  # Define threshold based on margin    
    # Get indices where values are within the threshold
    min_indices = np.where(arr <= threshold)[0]
    # Identify consecutive sequences
    longest_sequence = None
    if len(min_indices) > 0:
        start = min_indices[0]
        max_duration = 0  # Track longest duration        
        for i in range(1, len(min_indices)):
            if min_indices[i] != min_indices[i - 1] + 1:  # Not consecutive
                duration = min_indices[i - 1] - start + 1
                if duration > max_duration:
                    max_duration = duration
                    longest_sequence = (start, min_indices[i - 1], duration)
                start = min_indices[i]  # Reset start index        
        # Check last detected sequence
        duration = min_indices[-1] - start + 1
        if duration > max_duration:
            longest_sequence = (start, min_indices[-1], duration)
    return min_val, threshold, longest_sequence


def resample_matrix(data, orig_rate, target_rate, axis=0):
    if orig_rate == target_rate:
        return data.copy()
    # Compute integer up/down factors using GCD
    up = int(target_rate)
    down = int(orig_rate)
    factor = gcd(up, down)
    up //= factor
    down //= factor
    return resample_poly(data, up=up, down=down, axis=axis)

def calculate_relative_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def calculate_distance_run(x_coords, y_coords):
    distances = np.sqrt(np.diff(x_coords) ** 2 + np.diff(y_coords) ** 2)
    for i in range(1, len(distances) - 1):
        if np.isnan(distances[i]):
            neighbors = [distances[i-1], distances[i+1]]
            distances[i] = np.mean([x for x in neighbors if not np.isnan(x)])
    total_distance_cm = np.nansum(distances) / pixel_to_cm  # Convert to cm
    return total_distance_cm, distances

def find_long_non_nan_sequences(arr, min_length=100):
    mask = ~np.isnan(arr)  # True for non-NaN values
    diff = np.diff(np.concatenate(([0], mask.astype(int), [0])))  # Add padding to detect edges
    starts = np.where(diff == 1)[0]  # Where a sequence starts
    ends = np.where(diff == -1)[0]   # Where a sequence ends
    sequences = [arr[start:end] for start, end in zip(starts, ends) if (end - start) > min_length]
    return sequences

def remove_outliers_median_filter(data, window=1):
    data = np.array(data, dtype=float)  # Ensure NumPy array with float type
    filtered_data = np.copy(data)  # Copy to avoid modifying original data
    half_window = window // 2
    for i in range(len(data)):
        # Define window range, ensuring it doesn't exceed bounds
        start = max(0, i - half_window)
        end = min(len(data), i + half_window + 1)
        # Extract local values in window
        local_values = data[start:end]
        # Check if the window contains at least one non-NaN value
        if np.all(np.isnan(local_values)):
            median_value = np.nan  # Keep NaN if no valid numbers
        else:
            median_value = np.nanmedian(local_values)  # Compute median ignoring NaNs
        # Replace only if the current value is not NaN
        if not np.isnan(data[i]):
            filtered_data[i] = median_value
    return filtered_data

def replace_high_speed_points_with_nan(x, y, speed_threshold):
    x = np.array(x, dtype='float')
    y = np.array(y, dtype='float')
    # Compute speed between consecutive points
    dx = np.diff(x)
    dy = np.diff(y)
    speeds = np.sqrt(dx**2 + dy**2)
    # Create mask for speed exceeding threshold
    high_speed_mask = speeds > speed_threshold
    # We mark i+1 as NaN if speed between them is too high
    x_out = x.copy()
    y_out = y.copy()
    for i in range(len(high_speed_mask)):
        if high_speed_mask[i]:
            # Only mark the faster of the two points
            if i > 0 and i < len(x) - 1:
                if speeds[i] > speeds[i - 1]:
                    x_out[i + 1] = np.nan
                    y_out[i + 1] = np.nan
                else:
                    x_out[i] = np.nan
                    y_out[i] = np.nan
    return x_out, y_out

def interpolate_2d_path(x, y, kind='linear', fill='extrapolate'):
    x = np.array(x, dtype='float')
    y = np.array(y, dtype='float')
    indices = np.arange(len(x))
    valid_mask = ~np.isnan(x) & ~np.isnan(y)
    if np.sum(valid_mask) < 2:
        raise ValueError("Not enough valid points to interpolate/extrapolate.")
    interp_x = interp1d(indices[valid_mask], x[valid_mask], kind=kind, fill_value=fill, bounds_error=False)
    interp_y = interp1d(indices[valid_mask], y[valid_mask], kind=kind, fill_value=fill, bounds_error=False)
    x_filled = x.copy()
    y_filled = y.copy()
    nan_mask = np.isnan(x) | np.isnan(y)
    x_filled[nan_mask] = interp_x(indices[nan_mask])
    y_filled[nan_mask] = interp_y(indices[nan_mask])
    return x_filled, y_filled

def limit_speed(x, y, max_speed):
    dx = np.diff(x.copy())
    dy = np.diff(y.copy())
    speeds = np.sqrt(dx**2 + dy**2)
    for i,t in enumerate(speeds):
        if t > max_speed:        
            x[i+1] = x[i] 
            y[i+1] = y[i] 
            x[i+2] = x[i] 
            y[i+2] = y[i] 
    return x, y

def remove_short_sequences(arr, max_len=10):
    arr = np.array(arr, dtype='float')
    result = arr.copy()
    is_value = ~np.isnan(arr)
    i = 0
    while i < len(arr):
        if is_value[i]:
            start = i
            while i < len(arr) and is_value[i]:
                i += 1
            end = i
            seq_len = end - start
            # Check if surrounded by NaNs and short enough
            if seq_len <= max_len:
                left_nan = (start == 0) or np.isnan(arr[start - 1])
                right_nan = (end == len(arr)) or np.isnan(arr[end])  # safe for edge
                if left_nan and right_nan:
                    result[start:end] = np.nan
        else:
            i += 1
    return result

def Convert(string):
            li = list(string.split(", "))
            li2 = len(li)
            return li2

def marcenkopastur(significance):
    nbins = significance.nbins
    nneurons = significance.nneurons
    tracywidom = significance.tracywidom
    q = float(nbins)/float(nneurons)
    lambdaMax = pow((1+np.sqrt(1/q)),2)
    lambdaMax += tracywidom*pow(nneurons,-2./3)
    return lambdaMax

def getlambdacontrol(zactmat_):
    significance_ = PCA()
    significance_.fit(zactmat_.T)
    lambdamax_ = np.max(significance_.explained_variance_)
    return lambdamax_

def binshuffling(zactmat,significance):
    np.random.seed()
    lambdamax_ = np.zeros(significance.nshu)
    for shui in range(significance.nshu):
        zactmat_ = np.copy(zactmat)
        for (neuroni,activity) in enumerate(zactmat_):
            randomorder = np.argsort(np.random.rand(significance.nbins))
            zactmat_[neuroni,:] = activity[randomorder]
        lambdamax_[shui] = getlambdacontrol(zactmat_)
    lambdaMax = np.percentile(lambdamax_,significance.percentile)
    return lambdaMax

def circshuffling(zactmat,significance):
    np.random.seed()
    lambdamax_ = np.zeros(significance.nshu)
    for shui in range(significance.nshu):
        zactmat_ = np.copy(zactmat)
        for (neuroni,activity) in enumerate(zactmat_):
            cut = int(np.random.randint(significance.nbins*2))
            zactmat_[neuroni,:] = np.roll(activity,cut)
        lambdamax_[shui] = getlambdacontrol(zactmat_)
    lambdaMax = np.percentile(lambdamax_,significance.percentile)
    return lambdaMax

def runSignificance(zactmat,significance):
    if significance.nullhyp == 'mp':
        lambdaMax = marcenkopastur(significance)
    elif significance.nullhyp == 'bin':
        lambdaMax = binshuffling(zactmat,significance)
    elif significance.nullhyp == 'circ':
        lambdaMax = circshuffling(zactmat,significance)
    else:
        print('ERROR !')
        print('    nyll hypothesis method '+str(nullhyp)+' not understood')
        significance.nassemblies = np.nan
    nassemblies = np.sum(significance.explained_variance_>lambdaMax)
    significance.nassemblies = nassemblies
    return significance

def extractPatterns(actmat,significance,method):
    nassemblies = significance.nassemblies
    if method == 'pca':
        idxs = np.argsort(-significance.explained_variance_)[0:nassemblies]
        patterns = significance.components_[idxs,:]
    elif method == 'ica':
        from sklearn.decomposition import FastICA
        ica = FastICA(n_components=nassemblies, max_iter=1000)
        ica.fit(actmat.T)
        patterns = ica.components_
    else:
        print('ERROR !')
        print('    assembly extraction method '+str(method)+' not understood')
        patterns = np.nan
    if patterns is not np.nan:
        patterns = patterns.reshape(nassemblies,-1)
        norms = np.linalg.norm(patterns,axis=1)
        patterns /= np.matlib.repmat(norms,np.size(patterns,1),1).T
    return patterns

def runPatterns(actmat, method='ica', nullhyp = 'mp', nshu = 1000, percentile = 99, tracywidom = False):
    nneurons = np.size(actmat,0)
    nbins = np.size(actmat,1)
    silentneurons = np.var(actmat,axis=1)==0
    actmat_ = actmat[~silentneurons,:]
    zactmat_ = stats.zscore(actmat_,axis=1)
    significance = PCA()
    significance.fit(zactmat_.T)
    significance.nneurons = nneurons
    significance.nbins = nbins
    significance.nshu = nshu
    significance.percentile = percentile
    significance.tracywidom = tracywidom
    significance.nullhyp = nullhyp
    significance = runSignificance(zactmat_,significance)
    if np.isnan(significance.nassemblies):
        patterns = []
        zactmat = []
        significance = []
        #return
    if significance.nassemblies<1:
        print('WARNING 1!')
        print('    no assembly detecded!')
        patterns = []
        zactmat = []
        significance = []
    else:
        patterns_ = extractPatterns(zactmat_,significance,method)
        if patterns_ is np.nan:
            patterns = []
            zactmat = []
            significance = []
            #return
        patterns = np.zeros((np.size(patterns_,0),nneurons))
        patterns[:,~silentneurons] = patterns_
        zactmat = np.copy(actmat)
        zactmat[~silentneurons,:] = zactmat_
    return patterns,significance,zactmat

def computeAssemblyActivity(patterns,zactmat,zerodiag = True):
    if len(patterns) == 0:
        print('WARNING 2!')
        print('    no assembly detecded!')
        assemblyAct = []
    else:
        nassemblies = len(patterns)
        nbins = np.size(zactmat,1)
        assemblyAct = np.zeros((nassemblies,nbins))
        for (assemblyi,pattern) in enumerate(patterns):
            projMat = np.outer(pattern,pattern)
            projMat -= zerodiag*np.diag(np.diag(projMat))
            for bini in range(nbins):
                assemblyAct[assemblyi,bini] = np.dot(np.dot(zactmat[:,bini],projMat),zactmat[:,bini])
    return assemblyAct

# Conversion des quaternions en angles d'Euler
def quaternion_to_euler(qw, qx, qy, qz):
    sinr_cosp = 2 * (qw * qx + qy * qz)
    cosr_cosp = 1 - 2 * (qx * qx + qy * qy)
    roll = np.arctan2(sinr_cosp, cosr_cosp)

    sinp = 2 * (qw * qy - qz * qx)
    sinp = np.clip(sinp, -1.0, 1.0)
    pitch = np.arcsin(sinp)

    siny_cosp = 2 * (qw * qz + qx * qy)
    cosy_cosp = 1 - 2 * (qy * qy + qz * qz)
    yaw = np.arctan2(siny_cosp, cosy_cosp)

    return roll, pitch, yaw

def direction(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    if dx == 0 and dy == 0:
        return np.nan  # undefined direction
    angle_rad = math.atan2(dy, dx)
    angle_deg = math.degrees(angle_rad)
    return (angle_deg + 360) % 360

Load sleep score and Ca2+ time series numpy arrays

In [ ]:
# Get the current date and time
FolderNameSave=str(datetime.now())[:19]
FolderNameSave = FolderNameSave.replace(" ", "_").replace(".", "_").replace(":", "_")

#destination_folder= f"//10.69.168.1/crnldata/forgetting/Aurelie/MiniscopeOE_analysis/PlaceCells_experiment/Cheeseboard_{FolderNameSave}{AnalysisID}" 
#os.makedirs(destination_folder)
#folder_to_save=Path(destination_folder)

AllCellDict={}
AllTimeDict={}
AllCellDictSess={}
AllTimeDictSess={}

AllCellDict_HD={}
AllTimeDict_HD={}
AllCellDictSess_HD={}
AllTimeDictSess_HD={}

for dpath in Path(dir).glob('**/PlaceCells_experiment/mappingsAB.pkl'):

    mappfile = open(dpath.parents[0]/ f'mappingsAB.pkl', 'rb')
    mapping = pickle.load(mappfile)
    mapping_sess = mapping['session']   
        
    centfile = open(dpath.parents[0]/ f'centsAB.pkl', 'rb')
    centroids = pickle.load(centfile) 

    mice = dpath.parents[1].parts[-1]
    NeuronType = dpath.parents[2].parts[-1]
    
    print(f"Processing mouse {mice} of type {NeuronType}")

    sess=0

    minian_folders = [f for f in dpath.parents[0].rglob('minian') if f.is_dir()]

    for minianpath in minian_folders: # for each minian folders found in this mouse 

        if any(p in all_expe_types for p in minianpath.parts): # have to be to the expe_types

            try: 
                session_path=minianpath.parents[1]        
                tsmini= pd.read_csv(list(session_path.glob('*V4_Miniscope/timeStamps.csv'))[0])['Time Stamp (ms)']
                V4subfolder=False
                session_type=minianpath.parents[2].name
                session_date=minianpath.parents[3].name
                session_time=minianpath.parents[1].name
                session=session_time
         
            except:
                session_path=minianpath.parents[2]      
                tsmini= pd.read_csv(list(session_path.glob('*V4_Miniscope/timeStamps.csv'))[0])['Time Stamp (ms)']                      
                V4subfolder=True
                session_type=minianpath.parents[3].name
                session_date=minianpath.parents[4].name
                session_time=minianpath.parents[0].name
                session=session_time
            print(f"Processing {session_type} session: {session} on the {session_date} ")

            # Minian data 
                        
            minian_ds = open_minian(minianpath)
            Co = minian_ds['C']  # calcium traces
            minian_freq=round(1/np.mean(np.diff(np.array(tsmini)/1000)))
            print('... miniscope sample rate =', minian_freq, 'Hz')            
            try: 
                TodropFile = minianpath / f'TodropFileAB.json'
                with open(TodropFile, 'r') as f:
                    unit_to_drop = json.load(f)
            except:
                TodropFile = minianpath.parent / f'TodropFileAB.json'
                with open(TodropFile, 'r') as f:
                    unit_to_drop = json.load(f)

            if V4subfolder:
                V4subfolder_id = int(minianpath.parent.name[-1]) - 1 
                ts_start = V4subfolder_id*15*1000 # cause 15 videos per subfolders and 1000 frames per videos
                ts_stop = np.shape(Co)[1] + ts_start
                tsmini_sub=tsmini[ts_start:ts_stop]
                tsmini_sub=tsmini_sub.reset_index(drop=True)    
            else:
                tsmini_sub=tsmini


            # Extraction des quaternions

            head_df= pd.read_csv(list(session_path.glob('*V4_Miniscope/headOrientation.csv'))[0])
            qw = head_df['qw'].to_numpy()
            qx = head_df['qx'].to_numpy()
            qy = head_df['qy'].to_numpy()
            qz = head_df['qz'].to_numpy()
            roll, pitch, yaw = quaternion_to_euler(qw, qx, qy, qz)
            roll_deg = np.degrees(roll) # nose goes clockwise or anticlockwise
            pitch_deg = np.degrees(pitch) # nose up or down
            yaw_deg = np.degrees(yaw) # nose moves from side to side                
            

            # DeepLabCut data

            dlcfile=''
            dlcpath=Path(f'{Path(session_path)}/My_First_WebCam/')
            for file in os.listdir(dlcpath):
                if file.endswith(('.h5')):
                    dlcfile=file
                    break
            dlc_path = os.path.join(dlcpath, dlcfile)
            df = pd.read_hdf(dlc_path)
            directory = os.path.dirname(dlc_path)
            timestamps_path = Path(directory,'timeStamps.csv')
            if timestamps_path.exists():
                timestamps = pd.read_csv(timestamps_path)
                tswebcam = np.array(timestamps['Time Stamp (ms)'])
                frame_rate = round(1/(np.mean(np.diff(timestamps.iloc[:,1]))/1000))  # fps
            else:
                frame_rate = 16  # fps /!\ CHANGE ACCORDING TO YOUR DATA
            
            # Head 

            df_head= df.copy()
            df_head.iloc[:, 0] = df_head.apply(lambda row: row.iloc[0] if row.iloc[2] > 0.5 else np.nan, axis=1)
            df_head.iloc[:, 1] = df_head.apply(lambda row: row.iloc[1] if row.iloc[2] > 0.5 else np.nan, axis=1)
            X = df_head.iloc[:, 0]
            Y = df_head.iloc[:, 1]        
            individual_xO= np.array(X.values) 
            individual_yO = np.array(Y.values)
            for i, x in enumerate(individual_xO):# Define when the mouse is on the cheeseboard (start)
                y = individual_yO[i]
                if calculate_relative_distance(x, y, table_center_x, table_center_y) >= table_radius:
                    individual_xO[i] = np.nan
                    individual_yO[i] = np.nan
            individual_xOO = remove_short_sequences(individual_xO, max_len=3)
            individual_yOO = remove_short_sequences(individual_yO, max_len=3)
            x_start = find_long_non_nan_sequences(individual_xOO)[0][0] # first value of the first long non nan sequence
            y_start = find_long_non_nan_sequences(individual_yOO)[0][0] # first value of the first long non nan sequence
            start_frame = np.where(individual_xOO == x_start)[0][0].item()
            
            individual_xOO[:start_frame]=np.nan # remove any path before the real start
            individual_yOO[:start_frame]=np.nan # remove any path before the real start
            individual_x1, individual_y1 = replace_high_speed_points_with_nan(individual_xOO, individual_yOO, speed_threshold=10)
            last_frame = len(individual_x1)
            individual_x2, individual_y2 = interpolate_2d_path(individual_x1[start_frame:last_frame], individual_y1[start_frame:last_frame], kind='nearest')
            individual_x3, individual_y3 = limit_speed(individual_x2, individual_y2, max_speed=20)
            individual_x = np.concatenate((individual_x1[:start_frame], individual_x3))
            individual_y = np.concatenate((individual_y1[:start_frame], individual_y3))
            
            # Tail 

            df_tail= df.copy()
            df_tail.iloc[:, 3] = df_tail.apply(lambda row: row.iloc[3] if row.iloc[5] > 0.5 else np.nan, axis=1)
            df_tail.iloc[:, 4] = df_tail.apply(lambda row: row.iloc[4] if row.iloc[5] > 0.5 else np.nan, axis=1)
            Xt = df_tail.iloc[:, 3]
            Yt = df_tail.iloc[:, 4]        
            tail_xO= np.array(Xt.values) 
            tail_yO = np.array(Yt.values)
            for i, x in enumerate(tail_xO):# Define when the mouse is on the cheeseboard (start)
                y = tail_yO[i]
                if calculate_relative_distance(x, y, table_center_x, table_center_y) >= table_radius:
                    tail_xO[i] = np.nan
                    tail_yO[i] = np.nan
            tail_xOO = remove_short_sequences(tail_xO, max_len=3)
            tail_yOO = remove_short_sequences(tail_yO, max_len=3)
            tail_xOO[:start_frame]=np.nan # remove any path before the real start
            tail_yOO[:start_frame]=np.nan # remove any path before the real start
            tail_x1, tail_y1 = replace_high_speed_points_with_nan(tail_xOO, tail_yOO, speed_threshold=10)
            tail_x2, tail_y2 = interpolate_2d_path(tail_x1[start_frame:last_frame], tail_y1[start_frame:last_frame], kind='nearest')
            tail_x3, tail_y3 = limit_speed(tail_x2, tail_y2, max_speed=20)
            tail_x = np.concatenate((tail_x1[:start_frame], tail_x3))
            tail_y = np.concatenate((tail_y1[:start_frame], tail_y3))
                        

            # Keep only crossregistered cells

            C_sel=Co.drop_sel(unit_id=unit_to_drop)
            indexMappList=mapping_sess[session]
            kept_uniq_unit_List=[]
            for unit in C_sel['unit_id'].values:
                indexMapp = np.where(indexMappList == unit)[0]
                kept_uniq_unit_List.append(str(indexMapp))
            nb_unit=len(C_sel['unit_id'].values)
            if nb_unit == 0:
                print(f'... no cells kept in the session: {session}')
                continue
            print(f'... {nb_unit} kept cells in the session')


            # If the miniscope recording started after the webcam recording, cut the webcam data
            if tsmini_sub.iloc[0] > tswebcam[start_frame]:
                Newstart_frame = np.where(tswebcam >= tsmini_sub.iloc[0].item())[0][1].item()
                print(f'... webcam data cut to match miniscope length, new start at frame {Newstart_frame} (instead of {start_frame})')
                start_frame = Newstart_frame 
            # If the miniscope recording is shorter than the webcam recording, cut the webcam data
            if tsmini_sub.iloc[-1] < tswebcam[last_frame-1]:
                Newlast_frame = np.where(tswebcam <= tsmini_sub.iloc[-1].item())[0][-1].item()
                print(f'... webcam data cut to match miniscope length, new end at frame {Newlast_frame} (instead of {last_frame})')
                last_frame = Newlast_frame
            


            # Align data

            x = individual_x[start_frame:last_frame]
            y = individual_y[start_frame:last_frame]
            xt = tail_x[start_frame:last_frame]
            yt = tail_y[start_frame:last_frame]
            angles_deg = np.array([direction(x1, y1, x2, y2) for x1, y1, x2, y2 in zip(xt, yt, x, y)])

            closest_start= find_closest_index_sorted(tsmini_sub, tswebcam[start_frame])
            closest_end= find_closest_index_sorted(tsmini_sub, tswebcam[last_frame-1])

            roll_deg_ = roll_deg[closest_start:closest_end]
            pitch_deg_ = pitch_deg[closest_start:closest_end]
            yaw_deg_ = yaw_deg[closest_start:closest_end]


            # Keep only crossregistered cells

            Carray = C_sel.to_numpy().T
            Calcium = pd.DataFrame(Carray.T, index=C_sel['unit_id'].values.tolist())            

            n = int(np.floor(2 * table_radius / square_size))
            for unit in range(nb_unit): 
                indexMapp = np.where(mapping_sess[session] == Calcium.index[unit])[0]
                if len(indexMapp)>0 : # The neuron needs to be in the cross-registration
                    unique_unit = f'{mice}{str(indexMapp[0])}'
                    sess_unit = f'{mice}{str(indexMapp[0])}_s{sess}_{unit}'
                    Carray_unit = Carray[:,unit]
                    nr_tot_act_biaised = defaultdict(int) # Neuron activity per square
                    counts = defaultdict(int) # Count visits per square
                    nr_tot_act_biaised_HD = np.zeros(len(bin_centers))
                    counts_HD = np.zeros(len(bin_centers))

                    for idx, (px, py, angle) in enumerate(zip(x, y, angles_deg)):
                        if np.sqrt((px - table_center_x)**2 + (py - table_center_y)**2) > table_radius:
                            continue  # skip points outside circle
                        ix = int(np.floor((px - (table_center_x - n/2 * square_size)) / square_size))
                        iy = int(np.floor((py - (table_center_y - n/2 * square_size)) / square_size))
                        closest_point = find_closest_index_sorted(tsmini_sub, tswebcam[start_frame+idx])
                        nr_tot_act_biaised[(ix, iy)] += Carray_unit[closest_point]
                        counts[(ix, iy)] += 1/frame_rate 

                        bin_idx = int(angle // bin_size)
                        nr_tot_act_biaised_HD[bin_idx] += Carray_unit[closest_point]
                        counts_HD[bin_idx] += 1                        

                    if unique_unit in AllCellDict:
                        AllCellDict[unique_unit][session] = nr_tot_act_biaised 
                        AllTimeDict[unique_unit][session] = counts

                        AllCellDict_HD[unique_unit][session] = nr_tot_act_biaised_HD 
                        AllTimeDict_HD[unique_unit][session] = counts_HD

                    else :
                        AllCellDict[unique_unit] = {}
                        AllCellDict[unique_unit][session] = nr_tot_act_biaised 
                        AllTimeDict[unique_unit] = {}
                        AllTimeDict[unique_unit][session] = counts
                        
                        AllCellDict_HD[unique_unit] = {}
                        AllCellDict_HD[unique_unit][session] = nr_tot_act_biaised_HD 
                        AllTimeDict_HD[unique_unit] = {}
                        AllTimeDict_HD[unique_unit][session] = counts_HD

                    AllCellDictSess[sess_unit] = nr_tot_act_biaised 
                    AllTimeDictSess[sess_unit] = counts
                    AllCellDictSess_HD[sess_unit] = nr_tot_act_biaised_HD 
                    AllTimeDictSess_HD[sess_unit] = counts_HD

            sess+=1

    print(f'{len(AllCellDict.keys())} unique cells found so far')  
    
#########################################################################################
                # Average activity map across sessions for each cell #
##########################################################################################

SumActDict={}
for uniquecell in AllCellDict.keys():
    sums = {}
    for subdict in AllCellDict[uniquecell].values():
        for key, value in subdict.items():
            sums[key] = sums.get(key, 0) + value
    SumActDict[uniquecell] = {key: sums[key] for key in sums}

SumTimeDict={}
for uniquecell in AllTimeDict.keys():
    sums = {}
    for subdict in AllTimeDict[uniquecell].values():
        for key, value in subdict.items():
            sums[key] = sums.get(key, 0) + value
    SumTimeDict[uniquecell] = {key: sums[key] for key in sums}



SumActDict_HD={}
for outer_key, inner_dict in AllCellDict_HD.items():
    first_array = next(iter(inner_dict.values()))
    sum_array = np.zeros_like(first_array)
    for arr in inner_dict.values():
        sum_array += arr
    SumActDict_HD[outer_key] = sum_array


SumTimeDict_HD={}
for outer_key, inner_dict in AllTimeDict_HD.items():
    first_array = next(iter(inner_dict.values()))
    sum_array = np.zeros_like(first_array)
    for arr in inner_dict.values():
        sum_array += arr
    SumTimeDict_HD[outer_key] = sum_array


print(f'Total unique cell = {len(SumActDict.keys())}')  


# Place Cells

Plot place map of cell per session

In [ ]:
sorted_keys = sorted(AllCellDictSess.keys())
AllCellDictSess = {key: AllCellDictSess[key]for rank, key in enumerate(sorted_keys)}

nb_subplot=len(AllCellDictSess.keys())
rows = int(np.ceil(np.sqrt(nb_subplot)))  # Rows: ceil(sqrt(X))
cols = int(np.ceil(np.sqrt(nb_subplot)))  # Columns: floor(sqrt(X))

# Create the figure with a 2x2 grid
fig, axs = plt.subplots(rows, cols, figsize=(int(nb_subplot/10), int(nb_subplot/10)))
axs = axs.flatten()
plt.tight_layout()

sigma = 1  # Standard deviation for Gaussian kernel
max_row = int(table_radius*2/square_size)+1
max_col = int(table_radius*2/square_size)+1

Smoothed_ActSess={}
Smoothed_TimeSess={}
Spatial_infoSess = {}

for nsubplot, nr in enumerate(AllCellDictSess.keys()):

    nr_tot_act = AllCellDictSess[nr]    
    nr_tot_act_array = [[None for _ in range(max_col)] for _ in range(max_row)] # Initialize 2D array
    for (row, col), value in nr_tot_act.items(): # Fill array
        nr_tot_act_array[row][col] = value    
    nr_tot_act_array = [[np.nan if v is None else v for v in row] for row in nr_tot_act_array] # Replace None with np.nan
    nr_tot_act_array = np.array(nr_tot_act_array, dtype=float)
    nr_tot_act_array=nr_tot_act_array.T

    array_for_filter = np.nan_to_num(nr_tot_act_array, nan=0.0) # Replace nan with 0 for Gaussian filtering
    act_smoothed_array = gaussian_filter(array_for_filter, sigma=sigma)    # Apply 2D Gaussian filter


    nr_tot_time = AllTimeDictSess[nr]
    nr_tot_time_array = [[None for _ in range(max_col)] for _ in range(max_row)] # Initialize 2D array
    for (row, col), value in nr_tot_time.items():# Fill array
        nr_tot_time_array[row][col] = value
    nr_tot_time_array = [[np.nan if v is None else v for v in row] for row in nr_tot_time_array] # Replace None with np.nan
    nr_tot_time_array = np.array(nr_tot_time_array, dtype=float)
    nr_tot_time_array=nr_tot_time_array.T

    array_for_filter = np.nan_to_num(nr_tot_time_array, nan=0.0) # Replace nan with 0 for Gaussian filtering
    time_smoothed_array = gaussian_filter(array_for_filter, sigma=sigma)    # Apply 2D Gaussian filter
    time_smoothed_array[time_smoothed_array < .5] = np.nan  # do not consider if spent less than 0.1 second in the square

    # Ratio
    act_smoothed_array = np.divide(act_smoothed_array, time_smoothed_array)

    # --- Mask outside circle ---
    rows, cols = act_smoothed_array.shape
    center_row, center_col = rows // 2, cols // 2
    radius = min(rows, cols) // 2   
    Y, X = np.ogrid[:rows, :cols]# Create a circular mask
    dist_from_center = np.sqrt((X - center_col)**2 + (Y - center_row)**2)
    mask = dist_from_center < radius    
    act_smoothed_array_mask = np.where(mask, act_smoothed_array, np.nan)# Apply mask: set values outside circle to NaN
    Smoothed_ActSess[nr]= act_smoothed_array_mask

    time_smoothed_array_mask = np.where(mask, time_smoothed_array, np.nan)# Apply mask: set values outside circle to NaN
    Smoothed_TimeSess[nr]= time_smoothed_array_mask


    # --- Spatial information ---
    mean_act=np.nanmean(Smoothed_ActSess[nr])
    sum_time=np.nansum(Smoothed_TimeSess[nr])
    Spatial_infoSess[nr] = 0
    for rows in np.arange(np.shape(Smoothed_ActSess[nr])[0]) : 
        for cols in np.arange(np.shape(Smoothed_ActSess[nr])[1]) :             
            bin_act=Smoothed_ActSess[nr][rows,cols]
            p_bin=Smoothed_TimeSess[nr][rows,cols]/sum_time 
            if ~ np.isnan(bin_act): 
                if bin_act!=0:             
                    spatial_info_bin=p_bin*(bin_act/mean_act)*math.log2(bin_act/mean_act)
                else:
                    spatial_info_bin=0    
                Spatial_infoSess[nr] += spatial_info_bin


    # Mask outside circle 
    rows, cols = np.full((max_row, max_col),1).shape
    center_row, center_col = rows // 2, cols // 2
    radius = min(rows, cols) // 2   
    Y, X = np.ogrid[:rows, :cols]# Create a circular mask
    dist_from_center = np.sqrt((X - center_col)**2 + (Y - center_row)**2)
    mask = dist_from_center < radius    
    masked_array = np.where(mask, np.full((max_row, max_col),1), np.nan)# Apply mask: set values outside circle to NaN
    
    # Draw map
    axs[nsubplot].imshow(masked_array, cmap='Greys_r', origin='upper')
    axs[nsubplot].imshow(act_smoothed_array_mask, cmap='jet', origin='upper')
    axs[nsubplot].set_aspect('equal')
    axs[nsubplot].set_title(f'{nr} ({np.round(Spatial_infoSess[nr], 1)})', pad=0, loc='left', fontsize=15)
    axs[nsubplot].set_title(f"{nr.split('_')[0]}_{nr.split('_')[1]}", pad=0, loc='left', fontsize=max(7,int(nb_subplot/20)))

        
    # Remove box (spines)
    for spine in axs[nsubplot].spines.values():
        spine.set_visible(False)
    axs[nsubplot].set_xticks([])  # No x-axis ticks
    axs[nsubplot].set_yticks([])  # No y-axis ticks

# Adjust layout to avoid clipping
fig.subplots_adjust(wspace=.1, hspace=.2)
plt.show()


Plot place map of all unique cells

In [ ]:
sorted_keys = sorted(SumActDict.keys())
SumActDict = {key: SumActDict[key]for rank, key in enumerate(sorted_keys)}

nb_subplot=len(SumActDict.keys())
rows = int(np.ceil(np.sqrt(nb_subplot)))  # Rows: ceil(sqrt(X))
cols = int(np.ceil(np.sqrt(nb_subplot)))  # Columns: floor(sqrt(X))

# Create the figure with a 2x2 grid
fig, axs = plt.subplots(rows, cols, figsize=(int(nb_subplot/10), int(nb_subplot/10)))
axs = axs.flatten()
plt.tight_layout()

sigma = 1  # Standard deviation for Gaussian kernel
max_row = int(table_radius*2/square_size)+1
max_col = int(table_radius*2/square_size)+1

Smoothed_Act={}
Smoothed_Time={}
Spatial_info = {}

for nsubplot, nr in enumerate(SumActDict.keys()):

    nr_tot_act = SumActDict[nr]    
    nr_tot_act_array = [[None for _ in range(max_col)] for _ in range(max_row)] # Initialize 2D array
    for (row, col), value in nr_tot_act.items():# Fill array
        nr_tot_act_array[row][col] = value    
    nr_tot_act_array = [[np.nan if v is None else v for v in row] for row in nr_tot_act_array] # Replace None with np.nan
    nr_tot_act_array = np.array(nr_tot_act_array, dtype=float)
    nr_tot_act_array=nr_tot_act_array.T

    array_for_filter = np.nan_to_num(nr_tot_act_array, nan=0.0) # Replace nan with 0 for Gaussian filtering
    act_smoothed_array = gaussian_filter(array_for_filter, sigma=sigma)    # Apply 2D Gaussian filter


    nr_tot_time = SumTimeDict[nr]
    nr_tot_time_array = [[None for _ in range(max_col)] for _ in range(max_row)] # Initialize 2D array
    for (row, col), value in nr_tot_time.items():# Fill array
        nr_tot_time_array[row][col] = value
    nr_tot_time_array = [[np.nan if v is None else v for v in row] for row in nr_tot_time_array] # Replace None with np.nan
    nr_tot_time_array = np.array(nr_tot_time_array, dtype=float)
    nr_tot_time_array=nr_tot_time_array.T

    array_for_filter = np.nan_to_num(nr_tot_time_array, nan=0.0) # Replace nan with 0 for Gaussian filtering
    time_smoothed_array = gaussian_filter(array_for_filter, sigma=sigma)    # Apply 2D Gaussian filter
    time_smoothed_array[time_smoothed_array < .5] = np.nan  # do not consider if spent less than 0.1 second in the square

    # Ratio
    act_smoothed_array = np.divide(act_smoothed_array, time_smoothed_array)

    # --- Mask outside circle ---
    rows, cols = act_smoothed_array.shape
    center_row, center_col = rows // 2, cols // 2
    radius = min(rows, cols) // 2   
    Y, X = np.ogrid[:rows, :cols]# Create a circular mask
    dist_from_center = np.sqrt((X - center_col)**2 + (Y - center_row)**2)
    mask = dist_from_center < radius    
    act_smoothed_array_mask = np.where(mask, act_smoothed_array, np.nan)# Apply mask: set values outside circle to NaN
    Smoothed_Act[nr]= act_smoothed_array_mask

    time_smoothed_array_mask = np.where(mask, time_smoothed_array, np.nan)# Apply mask: set values outside circle to NaN
    Smoothed_Time[nr]= time_smoothed_array_mask


    # --- Spatial information ---
    mean_act=np.nanmean(Smoothed_Act[nr])
    sum_time=np.nansum(Smoothed_Time[nr])
    Spatial_info[nr] = 0
    for rows in np.arange(np.shape(Smoothed_Act[nr])[0]) : 
        for cols in np.arange(np.shape(Smoothed_Act[nr])[1]) :             
            bin_act=Smoothed_Act[nr][rows,cols]
            p_bin=Smoothed_Time[nr][rows,cols]/sum_time 
            if ~ np.isnan(bin_act): 
                if bin_act!=0:             
                    spatial_info_bin=p_bin*(bin_act/mean_act)*math.log2(bin_act/mean_act)
                else:
                    spatial_info_bin=0    
                Spatial_info[nr] += spatial_info_bin


    # Mask outside circle 
    rows, cols = np.full((max_row, max_col),1).shape
    center_row, center_col = rows // 2, cols // 2
    radius = min(rows, cols) // 2   
    Y, X = np.ogrid[:rows, :cols]# Create a circular mask
    dist_from_center = np.sqrt((X - center_col)**2 + (Y - center_row)**2)
    mask = dist_from_center < radius    
    masked_array = np.where(mask, np.full((max_row, max_col),1), np.nan)# Apply mask: set values outside circle to NaN
    
    # Draw map
    axs[nsubplot].imshow(masked_array, cmap='Greys_r', origin='upper')
    axs[nsubplot].imshow(act_smoothed_array_mask, cmap='jet', origin='upper')
    axs[nsubplot].set_aspect('equal')
    axs[nsubplot].set_title(f'{nr} ({np.round(Spatial_info[nr], 1)})', pad=0, loc='left', fontsize=max(7,int(nb_subplot/10)))

        
    # Remove box (spines)
    for spine in axs[nsubplot].spines.values():
        spine.set_visible(False)
    axs[nsubplot].set_xticks([])  # No x-axis ticks
    axs[nsubplot].set_yticks([])  # No y-axis ticks

# Adjust layout to avoid clipping
fig.subplots_adjust(wspace=.1, hspace=.2)
plt.show()


# Head Direction Cells

Plot Head direction of cells per session

In [ ]:
sorted_keys = sorted(AllCellDictSess_HD.keys())
AllCellDictSess_HD = {key: AllCellDictSess_HD[key]for rank, key in enumerate(sorted_keys)}

nb_subplot=len(AllCellDictSess_HD.keys())
rows = int(np.ceil(np.sqrt(nb_subplot)))  # Rows: ceil(sqrt(X))
cols = int(np.ceil(np.sqrt(nb_subplot)))  # Columns: floor(sqrt(X))

# Create the figure with a 2x2 grid
fig, axs = plt.subplots(rows, cols, figsize=(int(nb_subplot/10), int(nb_subplot/10)),  subplot_kw={'projection': 'polar'})
axs = axs.flatten()
plt.tight_layout()

bin_size = 10
bin_edges = np.arange(0, 361, bin_size)
bin_centers = np.radians(bin_edges[:-1] + bin_size/2)

HeadDirection_infoSess={}
for nsubplot, nr in enumerate(AllCellDictSess_HD.keys()):

    ax = axs[nsubplot]
    ax.set_theta_zero_location("E")  # 0° at East
    ax.set_theta_direction(-1)        # Clockwise
    ax.set_title(f"{nr.split('_')[0]}_{nr.split('_')[1]}", pad=0, loc='left', fontsize=max(7,int(nb_subplot/10)))
    HeadDirection_infoSess[nr] = np.divide(AllCellDictSess_HD[nr], AllTimeDictSess_HD[nr])
    
    
    #ax.bar(bin_centers, HeadDirection_infoSess[nr], width=np.radians(bin_size), alpha=0.6, color= 'k')   
    
    theta = bin_centers
    theta = np.append(theta, theta[0])   
    r = HeadDirection_infoSess[nr]
    r_smooth = gaussian_filter1d(r, sigma=1, mode='wrap')  # wrap for circular smoothing
    r_smooth = np.append(r_smooth, r_smooth[0])
    pref_idx = np.argmax(r_smooth)    # Find preferred angle
    pref_angle = theta[pref_idx]    
    color = plt.cm.hsv(pref_angle / (2*np.pi))# pref_angle ranges 0 to 2*pi -> normalize to 0-1 for colormap
    ax.plot(theta, r_smooth, color=color)
    
    #r = np.append(r, r[0])
    #ax.plot(theta, r, linestyle='-', alpha=0.7, color= 'r')

    ax.set_xticks([])
    ax.set_yticks([])

# Adjust layout to avoid clipping
fig.subplots_adjust(wspace=.1, hspace=.2)
plt.show()

Plot Head direction of all unique cells

In [ ]:
sorted_keys = sorted(SumActDict_HD.keys())
SumActDict_HD = {key: SumActDict_HD[key]for rank, key in enumerate(sorted_keys)}

nb_subplot=len(SumActDict_HD.keys())
rows = int(np.ceil(np.sqrt(nb_subplot)))  # Rows: ceil(sqrt(X))
cols = int(np.ceil(np.sqrt(nb_subplot)))  # Columns: floor(sqrt(X))

# Create the figure with a 2x2 grid
fig, axs = plt.subplots(rows, cols, figsize=(int(nb_subplot/10), int(nb_subplot/10)),  subplot_kw={'projection': 'polar'})
axs = axs.flatten()
plt.tight_layout()

bin_size = 10
bin_edges = np.arange(0, 361, bin_size)
bin_centers = np.radians(bin_edges[:-1] + bin_size/2)

HeadDirection_info={}
for nsubplot, nr in enumerate(SumActDict_HD.keys()):

    ax = axs[nsubplot]
    ax.set_theta_zero_location("E")  # 0° at East
    ax.set_theta_direction(-1)        # Clockwise
    ax.set_title(f'{nr}', pad=0, loc='left', fontsize=max(7,int(nb_subplot/10)))
    HeadDirection_info[nr] = np.divide(SumActDict_HD[nr], SumTimeDict_HD[nr])
    
    #ax.bar(bin_centers, HeadDirection_info[nr], width=np.radians(bin_size), alpha=0.6, color= 'k')   
    
    theta = bin_centers
    theta = np.append(theta, theta[0])  
    r = HeadDirection_info[nr]
    r_smooth = gaussian_filter1d(r, sigma=1, mode='wrap')  # wrap for circular smoothing
    r_smooth = np.append(r_smooth, r_smooth[0])
    pref_idx = np.argmax(r_smooth)    # Find preferred angle
    pref_angle = theta[pref_idx]    
    color = plt.cm.hsv(pref_angle / (2*np.pi))# pref_angle ranges 0 to 2*pi -> normalize to 0-1 for colormap
    ax.plot(theta, r_smooth, color=color)
    
    #r = np.append(r, r[0])
    #ax.plot(theta, r, linestyle='-', alpha=0.7, color= 'r')
    
    ax.set_xticks([])
    ax.set_yticks([])

fig.subplots_adjust(wspace=.1, hspace=.2)
plt.show()

# Run permutation tests on circularly shifted calcium traces

In [ ]:
ShuffledSpatial_info={}
ShuffledHeadDirection_info={}
for nr in SumActDict.keys():
    ShuffledSpatial_info[nr]={}
    ShuffledHeadDirection_info[nr]={}

nb_tot_permutation=1000

for permutation_nb in range(nb_tot_permutation):

    AllCellDict_Sh={}
    AllTimeDict_Sh={}
    AllCellDictSess_Sh={}
    AllTimeDictSess_Sh={}

    AllCellDict_HD_Sh={}
    AllTimeDict_HD_Sh={}
    AllCellDictSess_HD_Sh={}
    AllTimeDictSess_HD_Sh={}

    for dpath in Path(dir).glob('**/mappingsAB.pkl'):

        mappfile = open(dpath.parents[0]/ f'mappingsAB.pkl', 'rb')
        mapping = pickle.load(mappfile)
        mapping_sess = mapping['session']   
        mice = dpath.parents[1].parts[-1]
        NeuronType = dpath.parents[2].parts[-1]

        nb_minian_total=0

        minian_folders = [f for f in dpath.parents[0].rglob('minian') if f.is_dir()]

        for sess, minianpath in enumerate(minian_folders): # for each minian folders found in this mouse 

            if any(p in all_expe_types for p in minianpath.parts): # have to be to the expe_types

                session_type=minianpath.parents[2].name
                session_date=minianpath.parents[3].name
                session_time=minianpath.parents[1].name
                session=session_time
                session_path=minianpath.parents[1] 

                # Minian data
                
                minian_ds = open_minian(minianpath)
                Co = minian_ds['C']  # calcium traces
                tsmini= pd.read_csv(list(session_path.glob('*V4_Miniscope/timeStamps.csv'))[0])['Time Stamp (ms)']
                minian_freq=round(1/np.mean(np.diff(np.array(tsmini)/1000)))
                try: 
                    TodropFile = minianpath / f'TodropFileAB.json'
                    with open(TodropFile, 'r') as f:
                        unit_to_drop = json.load(f)
                except:
                    TodropFile = minianpath.parent / f'TodropFileAB.json'
                    with open(TodropFile, 'r') as f:
                        unit_to_drop = json.load(f)


            # DeepLabCut data

            dlcfile=''
            dlcpath=Path(f'{Path(session_path)}/My_First_WebCam/')
            for file in os.listdir(dlcpath):
                if file.endswith(('.h5')):
                    dlcfile=file
                    break
            dlc_path = os.path.join(dlcpath, dlcfile)
            df = pd.read_hdf(dlc_path)
            directory = os.path.dirname(dlc_path)
            timestamps_path = Path(directory,'timeStamps.csv')
            if timestamps_path.exists():
                timestamps = pd.read_csv(timestamps_path)
                tswebcam = np.array(timestamps['Time Stamp (ms)'])
                frame_rate = round(1/(np.mean(np.diff(timestamps.iloc[:,1]))/1000))  # fps
            else:
                frame_rate = 16  # fps /!\ CHANGE ACCORDING TO YOUR DATA
            
            
            # Head
            df_head= df.copy()
            df_head.iloc[:, 0] = df_head.apply(lambda row: row.iloc[0] if row.iloc[2] > 0.5 else np.nan, axis=1)
            df_head.iloc[:, 1] = df_head.apply(lambda row: row.iloc[1] if row.iloc[2] > 0.5 else np.nan, axis=1)
            X = df_head.iloc[:, 0]
            Y = df_head.iloc[:, 1]        
            individual_xO= np.array(X.values) 
            individual_yO = np.array(Y.values)
            for i, x in enumerate(individual_xO):# Define when the mouse is on the cheeseboard (start)
                y = individual_yO[i]
                if calculate_relative_distance(x, y, table_center_x, table_center_y) >= table_radius:
                    individual_xO[i] = np.nan
                    individual_yO[i] = np.nan
            individual_xOO = remove_short_sequences(individual_xO, max_len=3)
            individual_yOO = remove_short_sequences(individual_yO, max_len=3)
            x_start = find_long_non_nan_sequences(individual_xOO)[0][0] # first value of the first long non nan sequence
            y_start = find_long_non_nan_sequences(individual_yOO)[0][0] # first value of the first long non nan sequence
            start_frame = np.where(individual_xOO == x_start)[0][0].item()
            individual_xOO[:start_frame]=np.nan # remove any path before the real start
            individual_yOO[:start_frame]=np.nan # remove any path before the real start
            individual_x1, individual_y1 = replace_high_speed_points_with_nan(individual_xOO, individual_yOO, speed_threshold=10)
            last_frame = len(individual_x1)
            individual_x2, individual_y2 = interpolate_2d_path(individual_x1[start_frame:last_frame], individual_y1[start_frame:last_frame], kind='nearest')
            individual_x3, individual_y3 = limit_speed(individual_x2, individual_y2, max_speed=20)
            individual_x = np.concatenate((individual_x1[:start_frame], individual_x3))
            individual_y = np.concatenate((individual_y1[:start_frame], individual_y3))


            # Tail 
            df_tail= df.copy()
            df_tail.iloc[:, 3] = df_tail.apply(lambda row: row.iloc[3] if row.iloc[5] > 0.5 else np.nan, axis=1)
            df_tail.iloc[:, 4] = df_tail.apply(lambda row: row.iloc[4] if row.iloc[5] > 0.5 else np.nan, axis=1)
            Xt = df_tail.iloc[:, 3]
            Yt = df_tail.iloc[:, 4]        
            tail_xO= np.array(Xt.values) 
            tail_yO = np.array(Yt.values)
            for i, x in enumerate(tail_xO):# Define when the mouse is on the cheeseboard (start)
                y = tail_yO[i]
                if calculate_relative_distance(x, y, table_center_x, table_center_y) >= table_radius:
                    tail_xO[i] = np.nan
                    tail_yO[i] = np.nan
            tail_xOO = remove_short_sequences(tail_xO, max_len=3)
            tail_yOO = remove_short_sequences(tail_yO, max_len=3)
            tail_xOO[:start_frame]=np.nan # remove any path before the real start
            tail_yOO[:start_frame]=np.nan # remove any path before the real start
            tail_x1, tail_y1 = replace_high_speed_points_with_nan(tail_xOO, tail_yOO, speed_threshold=10)
            tail_x2, tail_y2 = interpolate_2d_path(tail_x1[start_frame:last_frame], tail_y1[start_frame:last_frame], kind='nearest')
            tail_x3, tail_y3 = limit_speed(tail_x2, tail_y2, max_speed=20)
            tail_x = np.concatenate((tail_x1[:start_frame], tail_x3))
            tail_y = np.concatenate((tail_y1[:start_frame], tail_y3))
            

            # Keep only crossregistered cells
            C_sel=Co.drop_sel(unit_id=unit_to_drop)
            indexMappList=mapping_sess[session]
            kept_uniq_unit_List=[]
            for unit in C_sel['unit_id'].values:
                indexMapp = np.where(indexMappList == unit)[0]
                kept_uniq_unit_List.append(str(indexMapp))
            nb_unit=len(C_sel['unit_id'].values)
            if nb_unit == 0:
                continue


            # If the miniscope recording started after the webcam recording, cut the webcam data
            if tsmini.iloc[0] > tswebcam[start_frame]:
                Newstart_frame = np.where(tswebcam >= tsmini.iloc[0].item())[0][1].item()
                start_frame = Newstart_frame 
            # If the miniscope recording is shorter than the webcam recording, cut the webcam data
            if tsmini.iloc[-1] < tswebcam[last_frame-1]:
                Newlast_frame = np.where(tswebcam <= tsmini.iloc[-1].item())[0][-1].item()
                last_frame = Newlast_frame
            

            # Align data
            x = individual_x[start_frame:last_frame]
            y = individual_y[start_frame:last_frame]   
            xt = tail_x[start_frame:last_frame]
            yt = tail_y[start_frame:last_frame]
            angles_deg = np.array([direction(x1, y1, x2, y2) for x1, y1, x2, y2 in zip(xt, yt, x, y)])
                            
            Carray = C_sel.to_numpy().T

            # Randomly shift the calcium trace to break temporal correlation with behavior
            offset = random.randint(20*minian_freq, len(Carray)) # Random shift between 20s and the length of the session
            Carray = np.roll(Carray, shift=offset, axis=0) 

            Calcium = pd.DataFrame(Carray.T, index=C_sel['unit_id'].values.tolist())            


            # Keep only crossregistered cells
            n = int(np.floor(2 * table_radius / square_size))
            for unit in range(nb_unit): 
                indexMapp = np.where(mapping_sess[session] == Calcium.index[unit])[0]
                if len(indexMapp)>0 : # The neuron needs to be in the cross-registration
                    unique_unit = f'{mice}{str(indexMapp[0])}'
                    sess_unit = f'{mice}{str(indexMapp[0])}_s{sess}_{unit}'
                    Carray_unit = Carray[:,unit]
                    nr_tot_act_biaised = defaultdict(int) # Neuron activity per square
                    counts = defaultdict(int) # Count visits per square
                    
                    nr_tot_act_biaised_HD = np.zeros(len(bin_centers))
                    counts_HD = np.zeros(len(bin_centers))

                    for idx, (px, py, angle) in enumerate(zip(x, y, angles_deg)):
                        if np.sqrt((px - table_center_x)**2 + (py - table_center_y)**2) > table_radius:
                            continue  # skip points outside circle
                        ix = int(np.floor((px - (table_center_x - n/2 * square_size)) / square_size))
                        iy = int(np.floor((py - (table_center_y - n/2 * square_size)) / square_size))
                        closest_point = find_closest_index_sorted(tsmini, tswebcam[start_frame+idx])
                        nr_tot_act_biaised[(ix, iy)] += Carray_unit[closest_point]
                        counts[(ix, iy)] += 1/frame_rate 

                        bin_idx = int(angle // bin_size)
                        nr_tot_act_biaised_HD[bin_idx] += Carray_unit[closest_point]
                        counts_HD[bin_idx] += 1
                        

                    if unique_unit in AllCellDict_Sh:
                        AllCellDict_Sh[unique_unit][session] = nr_tot_act_biaised 
                        AllTimeDict_Sh[unique_unit][session] = counts

                        AllCellDict_HD_Sh[unique_unit][session] = nr_tot_act_biaised_HD 
                        AllTimeDict_HD_Sh[unique_unit][session] = counts_HD

                    else :
                        AllCellDict_Sh[unique_unit] = {}
                        AllCellDict_Sh[unique_unit][session] = nr_tot_act_biaised 
                        AllTimeDict_Sh[unique_unit] = {}
                        AllTimeDict_Sh[unique_unit][session] = counts
                        
                        AllCellDict_HD_Sh[unique_unit] = {}
                        AllCellDict_HD_Sh[unique_unit][session] = nr_tot_act_biaised_HD 
                        AllTimeDict_HD_Sh[unique_unit] = {}
                        AllTimeDict_HD_Sh[unique_unit][session] = counts_HD

                    AllCellDictSess_Sh[sess_unit] = nr_tot_act_biaised 
                    AllTimeDictSess_Sh[sess_unit] = counts
                    AllCellDictSess_HD_Sh[sess_unit] = nr_tot_act_biaised_HD 
                    AllTimeDictSess_HD_Sh[sess_unit] = counts_HD

            nb_minian_total+=1

    #########################################################################################
                    # Average activity map across sessions for each cell #
    ##########################################################################################

    SumActDict_Sh={}
    for uniquecell in AllCellDict_Sh.keys():
        sums = {}
        for subdict in AllCellDict_Sh[uniquecell].values():
            for key, value in subdict.items():
                sums[key] = sums.get(key, 0) + value
        SumActDict_Sh[uniquecell] = {key: sums[key] for key in sums}

    SumTimeDict_Sh={}
    for uniquecell in AllTimeDict_Sh.keys():
        sums = {}
        for subdict in AllTimeDict_Sh[uniquecell].values():
            for key, value in subdict.items():
                sums[key] = sums.get(key, 0) + value
        SumTimeDict_Sh[uniquecell] = {key: sums[key] for key in sums}

    
    SumActDict_HD_Sh={}
    for outer_key, inner_dict in AllCellDict_HD_Sh.items():
        first_array = next(iter(inner_dict.values()))
        sum_array = np.zeros_like(first_array)
        for arr in inner_dict.values():
            sum_array += arr
        SumActDict_HD_Sh[outer_key] = sum_array


    SumTimeDict_HD_Sh={}
    for outer_key, inner_dict in AllTimeDict_HD_Sh.items():
        first_array = next(iter(inner_dict.values()))
        sum_array = np.zeros_like(first_array)
        for arr in inner_dict.values():
            sum_array += arr
        SumTimeDict_HD_Sh[outer_key] = sum_array


    sorted_keys = sorted(SumActDict_Sh.keys())
    SumActDict_Sh = {key: SumActDict_Sh[key]for rank, key in enumerate(sorted_keys)}

    sigma = 1  # Standard deviation for Gaussian kernel
    max_row = int(table_radius*2/square_size)+1
    max_col = int(table_radius*2/square_size)+1

    Smoothed_Act_Sh={}
    Smoothed_Time_Sh={}
    
    for nsubplot, nr in enumerate(SumActDict_Sh.keys()):

        nr_tot_act = SumActDict_Sh[nr]    
        nr_tot_act_array = [[None for _ in range(max_col)] for _ in range(max_row)] # Initialize 2D array
        for (row, col), value in nr_tot_act.items():# Fill array
            nr_tot_act_array[row][col] = value    
        nr_tot_act_array = [[np.nan if v is None else v for v in row] for row in nr_tot_act_array] # Replace None with np.nan
        nr_tot_act_array = np.array(nr_tot_act_array, dtype=float)
        nr_tot_act_array=nr_tot_act_array.T

        array_for_filter = np.nan_to_num(nr_tot_act_array, nan=0.0) # Replace nan with 0 for Gaussian filtering
        act_smoothed_array = gaussian_filter(array_for_filter, sigma=sigma)    # Apply 2D Gaussian filter


        nr_tot_time = SumTimeDict_Sh[nr]
        nr_tot_time_array = [[None for _ in range(max_col)] for _ in range(max_row)] # Initialize 2D array
        for (row, col), value in nr_tot_time.items():# Fill array
            nr_tot_time_array[row][col] = value
        nr_tot_time_array = [[np.nan if v is None else v for v in row] for row in nr_tot_time_array] # Replace None with np.nan
        nr_tot_time_array = np.array(nr_tot_time_array, dtype=float)
        nr_tot_time_array=nr_tot_time_array.T

        array_for_filter = np.nan_to_num(nr_tot_time_array, nan=0.0) # Replace nan with 0 for Gaussian filtering
        time_smoothed_array = gaussian_filter(array_for_filter, sigma=sigma)    # Apply 2D Gaussian filter
        time_smoothed_array[time_smoothed_array < .5] = np.nan  # do not consider if spent less than 0.1 second in the square

        # Ratio
        act_smoothed_array = np.divide(act_smoothed_array, time_smoothed_array)

        # --- Mask outside circle ---
        rows, cols = act_smoothed_array.shape
        center_row, center_col = rows // 2, cols // 2
        radius = min(rows, cols) // 2   
        Y, X = np.ogrid[:rows, :cols]# Create a circular mask
        dist_from_center = np.sqrt((X - center_col)**2 + (Y - center_row)**2)
        mask = dist_from_center < radius    
        act_smoothed_array_mask = np.where(mask, act_smoothed_array, np.nan)# Apply mask: set values outside circle to NaN
        Smoothed_Act_Sh[nr]= act_smoothed_array_mask

        time_smoothed_array_mask = np.where(mask, time_smoothed_array, np.nan)# Apply mask: set values outside circle to NaN
        Smoothed_Time_Sh[nr]= time_smoothed_array_mask


        # --- Spatial information ---
        mean_act=np.nanmean(Smoothed_Act_Sh[nr])
        sum_time=np.nansum(Smoothed_Time_Sh[nr])

        ShuffledSpatial_info[nr][permutation_nb] = 0

        for rows in np.arange(np.shape(Smoothed_Act_Sh[nr])[0]) : 
            for cols in np.arange(np.shape(Smoothed_Act_Sh[nr])[1]) :             
                bin_act=Smoothed_Act_Sh[nr][rows,cols]
                p_bin=Smoothed_Time_Sh[nr][rows,cols]/sum_time 
                if ~ np.isnan(bin_act): 
                    if bin_act!=0:             
                        spatial_info_bin=p_bin*(bin_act/mean_act)*math.log2(bin_act/mean_act)
                    else:
                        spatial_info_bin=0 

                    ShuffledSpatial_info[nr][permutation_nb] += spatial_info_bin
        
        ShuffledHeadDirection_info[nr][permutation_nb] = np.divide(SumActDict_HD_Sh[nr], SumTimeDict_HD_Sh[nr])    

    print(f'Permutation n° {permutation_nb} done')  

Results of permutations

In [ ]:
sorted_keys = sorted(ShuffledSpatial_info.keys())
ShuffledSpatial_info = {key: ShuffledSpatial_info[key] for rank, key in enumerate(sorted_keys)}
nb_tot_permutation = 1000
counts_PC = 0
counts_HD = 0
counts_PCHD = 0
for nr in ShuffledSpatial_info.keys():
    count_higher=0
    for permutation_nb in range(nb_tot_permutation):
        if ShuffledSpatial_info[nr][permutation_nb]>=Spatial_info[nr]:
            count_higher+=1
    p_value=(count_higher+1)/(nb_tot_permutation+1) # +1 for the observed value, +1 for the total number of shuffling
    
    
    observed_psth = HeadDirection_info[nr]
    shuffled_psths = arr = list(ShuffledHeadDirection_info[nr].values())
    alpha = 0.01
    lower_bound = np.percentile(shuffled_psths, alpha / 2 * 100, axis=0)
    upper_bound = np.percentile(shuffled_psths, (1 - alpha / 2) * 100, axis=0)
    significant_bins = (observed_psth < lower_bound) | (observed_psth > upper_bound)

    if significant_bins.any() and p_value>0.05 :
        print(f'{nr} is a head direction cell !')
        counts_HD+=1
    elif p_value<0.05 and not significant_bins.any() :
        print(f'{nr} is a place cell !')
        counts_PC+=1
    elif p_value<0.05 and significant_bins.any():
        print(f'{nr} is a place cell and a head direction cell !')
        counts_PC+=1
        counts_HD+=1
        counts_PCHD+=1

print(f'{counts_PC}/{len(Spatial_info.keys())} are place cells')
print(f'{counts_HD}/{len(HeadDirection_info.keys())} are head direction cells')
print(f'{counts_PCHD}/{len(HeadDirection_info.keys())} are both place cells and head direction cells')


In [ ]:
nb_subplot=len(HeadDirection_info.keys())
rows = int(np.ceil(np.sqrt(nb_subplot)))  # Rows: ceil(sqrt(X))
cols = int(np.ceil(np.sqrt(nb_subplot)))  # Columns: floor(sqrt(X))

fig, axs = plt.subplots(rows, cols, figsize=(8, 8))
axs = axs.flatten()
plt.tight_layout()

for nsubplot, nr in enumerate(HeadDirection_info.keys()):

    # Compute the observed PSTH
    observed_psth = HeadDirection_info[nr]
    shuffled_psths = arr = list(ShuffledHeadDirection_info[nr].values())

    # Compute significance (e.g., p < 0.05)
    alpha = 0.05
    lower_bound = np.percentile(shuffled_psths, alpha / 2 * 100, axis=0)
    upper_bound = np.percentile(shuffled_psths, (1 - alpha / 2) * 100, axis=0)
    significant_bins05 = (observed_psth < lower_bound) | (observed_psth > upper_bound)

    alpha = 0.01
    lower_bound = np.percentile(shuffled_psths, alpha / 2 * 100, axis=0)
    upper_bound = np.percentile(shuffled_psths, (1 - alpha / 2) * 100, axis=0)
    significant_bins01 = (observed_psth < lower_bound) | (observed_psth > upper_bound)

    alpha = 0.001
    lower_bound = np.percentile(shuffled_psths, alpha / 2 * 100, axis=0)
    upper_bound = np.percentile(shuffled_psths, (1 - alpha / 2) * 100, axis=0)
    significant_bins001 = (observed_psth < lower_bound) | (observed_psth > upper_bound)

    ax = axs[nsubplot]
    time_bins = np.linspace(0, 360, int(360//bin_size))
    ax.bar(time_bins, observed_psth, color='blue', width=bin_size, alpha=0.5)
    ax.bar(time_bins, shuffled_psths[0], color='gray', width=bin_size, alpha=0.5)

    ax.scatter(
        time_bins[significant_bins05],
        observed_psth[significant_bins05],
        color='green',
        label='p<0.05',
    )
    ax.scatter(
        time_bins[significant_bins01],
        observed_psth[significant_bins01],
        color='blue',
        label='p<0.01',
    )
    ax.scatter(
        time_bins[significant_bins001],
        observed_psth[significant_bins001],
        color='red',
        label='p<0.001',
    )

    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(f'{nr}', pad=0, loc='left', fontsize=7)

fig.subplots_adjust(wspace=.1, hspace=.2)
plt.show()


In [ ]:
nb_subplot=len(HeadDirection_info.keys())
rows = int(np.ceil(np.sqrt(nb_subplot)))  # Rows: ceil(sqrt(X))
cols = int(np.ceil(np.sqrt(nb_subplot)))  # Columns: floor(sqrt(X))

# Create the figure with a 2x2 grid
fig, axs = plt.subplots(rows, cols, figsize=(8, 8),  subplot_kw={'projection': 'polar'})
axs = axs.flatten()
plt.tight_layout()

bin_size = 10
bin_edges = np.arange(0, 361, bin_size)
bin_centers = np.radians(bin_edges[:-1] + bin_size/2)

for nsubplot, nr in enumerate(HeadDirection_info.keys()):

    ax = axs[nsubplot]
    ax.set_theta_zero_location("E")  # 0° at East
    ax.set_theta_direction(-1)        # Clockwise
    ax.set_title(f'{nr}', pad=0, loc='left', fontsize=7)
    
    theta = bin_centers
    theta = np.append(theta, theta[0])  
    r_smooth = gaussian_filter1d(HeadDirection_info[nr], sigma=1, mode='wrap')  # wrap for circular smoothing
    r_smooth = np.append(r_smooth, r_smooth[0])
    pref_idx = np.argmax(r_smooth)    # Find preferred angle
    pref_angle = theta[pref_idx]    
    color = plt.cm.hsv(pref_angle / (2*np.pi))# pref_angle ranges 0 to 2*pi -> normalize to 0-1 for colormap
    ax.plot(theta, r_smooth, color=color)

    r_shuffled = np.mean(list(ShuffledHeadDirection_info[nr].values()),0)
    r_smooth = gaussian_filter1d(r_shuffled, sigma=1, mode='wrap')  # wrap for circular smoothing
    r_smooth = np.append(r_smooth, r_smooth[0])
    ax.plot(theta, r_smooth, color='grey')
    
    ax.set_xticks([])
    ax.set_yticks([])

fig.subplots_adjust(wspace=.1, hspace=.1)
plt.show()


Save Spatial & Head Direction info 

In [ ]:
with open(os.path.join(dpath.parent, f"ShuffledSpatialInfo_{mice}.pkl"), "wb") as f:
    pickle.dump(ShuffledSpatial_info, f)
with open(os.path.join(dpath.parent, f"SpatialInfo_{mice}.pkl"), "wb") as f:
    pickle.dump(Spatial_info, f)

with open(os.path.join(dpath.parent, f"ShuffledHeadDirectionInfo_{mice}.pkl"), "wb") as f:
    pickle.dump(ShuffledHeadDirection_info, f)
with open(os.path.join(dpath.parent, f"HeadDirectionInfo_{mice}.pkl"), "wb") as f:
    pickle.dump(HeadDirection_info, f)

Load Spatial & Head Direction info 

In [ ]:
dpath= '//10.69.168.1/crnldata/forgetting/Aurelie/MiniscopeOE_data/L2_3_mice/RL/PlaceCells_experiment/'
with open(f'{dpath}/ShuffledSpatialInfo_RL.pkl', "rb") as f:
    ShuffledSpatial_info = pickle.load(f)
with open(f'{dpath}/SpatialInfo_RL.pkl', "rb") as f:
    Spatial_info = pickle.load(f)
with open(f'{dpath}/ShuffledHeadDirectionInfo_RL.pkl', "rb") as f:
    ShuffledHeadDirection_info = pickle.load(f)
with open(f'{dpath}/HeadDirectionInfo_RL.pkl', "rb") as f:
    HeadDirection_info = pickle.load(f)